# Preparing the environment for running Garfield++ on Colab

This notebook shows how to install the necessary programs to create finite element field maps and import them to Garfield++ on Colab.

In [ ]:
!cat /etc/os-release

# ROOT installation

Note that here it is important to provide the correct Python executable and library so that the Python components are compiled for the Python3 version installed on Colab. This is done using the flags for example `-DPYTHON_EXECUTABLE` when running `cmake`

In [ ]:
!apt-get install libxpm-dev
!wget https://root.cern/download/root_v6.20.04.source.tar.gz
!mkdir root
!mv root_v6.20.04.source.tar.gz root
!cd root && tar -xvf root_v6.20.04.source.tar.gz
!cd root && mkdir build
!export LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu:$LD_LIBRARY_PATH
!cd root/build && cmake -Dpython3=ON -DPYTHON_EXECUTABLE=/usr/bin/python3 -DPYTHON_LIBRARY=/usr/lib/x86_64-linux-gnu/libpython3.6m.so -DPYTHON_INCLUDE_DIR=/usr/include/python3.6m -DCMAKE_INSTALL_PREFIX=`pwd`/../install ../root-6.20.04
!cd root/build && make -j 2
!cd root/build && make install

# To double-check ROOT configuration options:
#!cd root/build && cat ROOTConfig.cmake

Confirm that this worked by loading relevant libraries and importing `ROOT`

In [ ]:
import sys
sys.path.append("/content/root/install/lib")
import ctypes
ctypes.cdll.LoadLibrary('/content/root/install/lib/libCore.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libThread.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libImt.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libRIO.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libNet.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libTree.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libMathCore.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libMatrix.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libHist.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGeom.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libXMLIO.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGdml.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGraf.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGpad.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGraf3d.so')

import ROOT

## Garfield++ installation

In [ ]:
!git clone https://gitlab.cern.ch/garfield/garfieldpp.git

In [ ]:
!mkdir garfield
!mv garfieldpp garfield
!mkdir garfield/build
!export ROOTSYS="/content/root/install"
!export PATH="$ROOTSYS/bin:$PATH"
!export LD_LIBRARY_PATH="$ROOTSYS/lib:$LD_LIBRARY_PATH"
!cd garfield/build && cmake -DWITH_EXAMPLES=OFF -DCMAKE_INSTALL_PREFIX=`pwd`/../install ../garfieldpp
!cd garfield/build && make -j 2
!cd garfield/build && make install

Confirm that this worked by loading the Garfield shared library with `ROOT`. Note that `GARFIELD_HOME` must be set or there will be an error locating the HEED database.

In [ ]:
import os
os.environ["GARFIELD_HOME"] = "/content/garfield/garfieldpp"
ROOT.gSystem.Load("/content/garfield/install/lib/libGarfield.so")

## Gmsh installation

Here we download the binary directly

In [ ]:
!wget https://gmsh.info/bin/Linux/gmsh-4.5.6-Linux64.tgz

And un-tar it

In [ ]:
!tar -xvf gmsh-4.5.6-Linux64.tgz
!mv gmsh-4.5.6-Linux64 gmsh

Check to make sure it runs

In [ ]:
!./gmsh/bin/gmsh -version

## Elmer installation
Be sure to compile with MPI off, or the program tends to crash at the very end of execution.

In [ ]:
!apt-get install libblas-dev
!mkdir elmer
!cd elmer && git clone https://github.com/ElmerCSC/elmerfem.git
!cd elmer && mkdir build
!cd elmer/build && cmake -DWITH_MPI:BOOLEAN=FALSE -DCMAKE_INSTALL_PREFIX=`pwd`/../install ../elmerfem
!cd elmer/build && make
!cd elmer/build && make install

Try running an Elmer executable to make sure it worked

In [ ]:
os.environ["PATH"] += ":/content/elmer/install/bin"
!ElmerSolver

## Save binaries for faster setup

In [ ]:
!tar -czvf environ.tar.gz elmer/install root/install garfield/install garfield/garfieldpp gmsh/bin

In [ ]:
!ls -l environ.tar.gz